<a href="https://colab.research.google.com/github/denis989/x.com_api_posts/blob/main/FIMI_post_download_fork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Universal Setup Block for FIMI Twitter Project**

In [ ]:
# === Universal Setup Block for FIMI Twitter Project ===

# 1. Install Libraries
print("Installing necessary libraries...")
!pip install tweepy pandas numpy -q # Added numpy explicitly
print("Libraries installed.")

# 2. Import Modules
print("Importing modules...")
import tweepy
import pandas as pd
import numpy as np
import json
import os
import glob
import time
from google.colab import drive
from datetime import datetime, timezone, timedelta
import matplotlib.pyplot as plt # Keep for potential quick plots
import matplotlib.dates as mdates # Keep for potential quick plots
import re # For filename sanitization
import traceback # For detailed error logging
print("Modules imported.")

# 3. Mount Google Drive
print("Mounting Google Drive...")
try:
    drive.mount('/content/drive', force_remount=True)
    print("Google Drive mounted successfully.")
    drive_mounted = True
except Exception as e:
    print(f"Error mounting Google Drive: {e}")
    drive_mounted = False

# 4. Define Core Paths
BASE_DRIVE_PATH = None
KEYS_PATH = None
LOGS_PATH = None
ACTORS_BASE_DIR = None # Base for /actors/ structure

if drive_mounted:
    BASE_DRIVE_PATH = "/content/drive/My Drive/ENC/Data/" # Standardized
    KEYS_PATH = os.path.join(BASE_DRIVE_PATH, "keys")
    LOGS_PATH = os.path.join(BASE_DRIVE_PATH, "logs")
    ACTORS_BASE_DIR = os.path.join(BASE_DRIVE_PATH, "actors") # For structured output
    TOKEN_FILE_PATH = os.path.join(KEYS_PATH, "bearer_token.txt")
    print(f"Base data path: {BASE_DRIVE_PATH}")
    try:
        os.makedirs(LOGS_PATH, exist_ok=True)
        os.makedirs(ACTORS_BASE_DIR, exist_ok=True) # Ensure base actors dir exists
        print("Logs and base Actors directories ensured.")
    except Exception as e:
        print(f"Warning: Could not create core directories: {e}")
else:
    print("Skipping path definitions as Drive is not mounted.")

# 5. Function to Load Bearer Token
def load_bearer_token(file_path):
    if not file_path or not os.path.exists(file_path):
        print(f"Error: Token file not found at '{file_path}'."); return None
    try:
        with open(file_path, 'r') as f: bearer_token = f.read().strip()
        if not bearer_token: print(f"Error: Token file '{file_path}' is empty."); return None
        print("Bearer Token loaded successfully.")
        return bearer_token
    except Exception as e: print(f"An error occurred reading token: {e}"); return None

# 6. Load Token and Initialize Tweepy Client
bearer_token = None
client = None
if drive_mounted and TOKEN_FILE_PATH:
    bearer_token = load_bearer_token(TOKEN_FILE_PATH)
    if bearer_token:
        try:
            client = tweepy.Client(bearer_token=bearer_token, wait_on_rate_limit=True)
            print("Tweepy Client initialized (wait_on_rate_limit=True).")
        except Exception as e: print(f"Error initializing Tweepy Client: {e}"); client = None
    else: print("Bearer token not loaded. Client not initialized.")
else: print("Drive not mounted or token path not set. Client not initialized.")

# 7. Function to Write Logs
def write_log_json(log_data, log_directory):
    if not log_directory or not os.path.exists(log_directory):
        print(f"Error: Log directory '{log_directory}' not valid. Cannot write log."); return None
    try:
        ts_iso = log_data.get("run_start_timestamp_utc", datetime.now(timezone.utc).isoformat(timespec='seconds'))
        # Ensure Z is appended if not present from isoformat()
        if not ts_iso.endswith("Z"): ts_iso = ts_iso.split('+')[0] + "Z"

        ts_filename = ts_iso.replace(':','').replace('-','').replace('T','_').replace('Z','')
        script_name_safe = log_data.get("script_name", "unknown_script").replace('.ipynb','').replace(':','-')[:50] # Made safer
        log_filename = f"{ts_filename}_{script_name_safe}_run_log.json"
        log_filepath = os.path.join(log_directory, log_filename)
        with open(log_filepath, 'w', encoding='utf-8') as f:
            json.dump(log_data, f, ensure_ascii=False, indent=2) # Changed to indent=2
        print(f"Log successfully saved to: {log_filepath}")
        return log_filepath
    except Exception as e: print(f"Error writing log file: {e}"); return None

print("\n=== Setup Complete ===")
if client: print("✅ Client initialized and ready.")
else: print("❌ Client FAILED to initialize. Check Drive mount and token file.")
# === END OF SETUP BLOCK ===

Installing necessary libraries...
Libraries installed.
Importing modules...
Modules imported.
Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully.
Base data path: /content/drive/My Drive/ENC/Data/
Logs and base Actors directories ensured.
Bearer Token loaded successfully.
Tweepy Client initialized (wait_on_rate_limit=True).

=== Setup Complete ===
✅ Client initialized and ready.


**Advanced Tweet Collector with Estimation ver. 2**

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import json
import os
import glob
import time
from google.colab import drive
from datetime import datetime, timezone, timedelta, date as py_date
import re
import traceback
import math

import ipywidgets as widgets
from ipywidgets import VBox, HBox, Layout
from IPython.display import display, clear_output, HTML as IPHTML
from tqdm.notebook import tqdm

# --- UI Elements (Define status_html early) ---
status_html = widgets.HTML(value="<i>Status updates...</i>", layout=Layout(height='100px', overflow_y='auto', border='1px solid lightgrey', padding='5px', width='95%'))
_status_lines = [] # Buffer for general status messages
_token_status_line = "" # Dedicated line for token status
_current_active_token = None # Track which token is currently active

def update_status_display_fn(new_messages_list, is_token_status_update=False):
    global _status_lines, _token_status_line, status_html

    if is_token_status_update:
        if isinstance(new_messages_list, list): _token_status_line = new_messages_list[0]
        else: _token_status_line = new_messages_list
    else:
        if isinstance(new_messages_list, str): new_messages_list = [new_messages_list]
        _status_lines.extend(new_messages_list)
        _status_lines = _status_lines[-3:] # Keep last 3 general status lines

    combined_status_display = f"<b>Token Status:</b> {_token_status_line if _token_status_line else 'Initializing...'}\n<hr>" + "\n".join(_status_lines)
    status_html.value = "<pre style='white-space: pre-wrap; word-wrap: break-word;'>" + combined_status_display + "</pre>"

# --- Global Client and Path Definitions ---
if 'BASE_DRIVE_PATH' not in globals() or BASE_DRIVE_PATH is None:
    update_status_display_fn(["Warning: BASE_DRIVE_PATH not found. Fallback for paths."])
    try:
        drive.mount('/content/drive', force_remount=True)
        BASE_DRIVE_PATH = "/content/drive/My Drive/ENC/Data/"
        KEYS_PATH = os.path.join(BASE_DRIVE_PATH, "keys")
        LOGS_PATH = os.path.join(BASE_DRIVE_PATH, "logs")
        ACTORS_BASE_DIR = os.path.join(BASE_DRIVE_PATH, "actors")
        os.makedirs(LOGS_PATH, exist_ok=True); os.makedirs(ACTORS_BASE_DIR, exist_ok=True)
    except Exception as e:
        update_status_display_fn([f"Critical Error: Fallback path setup failed: {e}"])
        BASE_DRIVE_PATH = None; KEYS_PATH = None
        LOGS_PATH = "./local_logs"; ACTORS_BASE_DIR = "./local_actors_data"
        os.makedirs(LOGS_PATH, exist_ok=True); os.makedirs(ACTORS_BASE_DIR, exist_ok=True)

# --- Multi-Client Management ---
clients_manager = {}
current_client_index_global = -1

# Phase 4: Rate limit optimization variables
rate_limit_optimization_enabled = True
estimated_requests_per_task = {}
optimal_pause_duration = {}

def _get_token_status_string_fn():
    """Helper to generate the token status summary string with color coding."""
    global _current_active_token
    if not clients_manager: return "No tokens initialized."
    status_parts = []
    now_utc = datetime.now(timezone.utc)

    for token_id_str_val in sorted(clients_manager.keys()):
        info = clients_manager[token_id_str_val]

        # Determine color and status
        if token_id_str_val == _current_active_token:
            # Green for active token
            color = "#00AA00"
            # Show remaining requests for active token
            if 'limits' in info:
                # Check both types of limits
                search_limits = info['limits'].get('search_tweets', {})
                count_limits = info['limits'].get('counts', {})

                # Показываем оба лимита если доступны
                limits_str = []
                if search_limits.get('remaining') is not None:
                    limits_str.append(f"S:{search_limits.get('remaining', '?')}/{search_limits.get('limit', '?')}")
                if count_limits.get('remaining') is not None:
                    limits_str.append(f"C:{count_limits.get('remaining', '?')}/{count_limits.get('limit', '?')}")

                if limits_str:
                    status_text = f"Active ({', '.join(limits_str)})"
                else:
                    status_text = "Active"
            else:
                status_text = "Active"
        elif now_utc < info['rate_limited_until']:
            # Red for rate-limited token with countdown
            color = "#FF0000"
            wait_secs = (info['rate_limited_until'] - now_utc).total_seconds()
            status_text = f"Wait {int(wait_secs)}s"
        else:
            # Default color for available tokens
            color = "inherit"
            # Show limits for available tokens
            if 'limits' in info:
                search_limits = info['limits'].get('search_tweets', {})
                count_limits = info['limits'].get('counts', {})

                limits_str = []
                if search_limits.get('remaining') is not None:
                    limits_str.append(f"S:{search_limits.get('remaining', '?')}/{search_limits.get('limit', '?')}")
                if count_limits.get('remaining') is not None:
                    limits_str.append(f"C:{count_limits.get('remaining', '?')}/{count_limits.get('limit', '?')}")

                if limits_str:
                    status_text = f"Ready ({', '.join(limits_str)})"
                else:
                    status_text = "Ready"
            else:
                status_text = "Ready"

        # Make text bold for active token
        font_weight = "bold" if token_id_str_val == _current_active_token else "normal"
        status_parts.append(f'<span style="color: {color}; font-weight: {font_weight};">{token_id_str_val}: {status_text}</span>')

    return " | ".join(status_parts)

def calculate_optimal_pause_fn(endpoint_type, estimated_requests, tokens_info):
    """
    Phase 4.1: Calculate optimal pause duration based on rate limits.

    Args:
        endpoint_type: 'counts' or 'search'
        estimated_requests: Number of requests expected
        tokens_info: Dictionary with token rate limit information

    Returns:
        Optimal pause duration in seconds
    """
    if not rate_limit_optimization_enabled:
        return 0.5  # Reduced default pause

    # Aggregate available rate limits across all tokens
    total_remaining = 0
    earliest_reset = None
    available_tokens = 0

    now_utc = datetime.now(timezone.utc)

    for token_id, info in tokens_info.items():
        # Skip tokens that are rate-limited
        if now_utc < info['rate_limited_until']:
            continue

        available_tokens += 1

        if 'limits' in info and endpoint_type in info['limits']:
            limits = info['limits'][endpoint_type]
            remaining = limits.get('remaining', 0)
            reset_ts = limits.get('reset_ts', 0)

            if remaining > 0:
                total_remaining += remaining
                if reset_ts > 0:
                    reset_dt = datetime.fromtimestamp(reset_ts, timezone.utc)
                    if earliest_reset is None or reset_dt < earliest_reset:
                        earliest_reset = reset_dt

    # If no limit info available, use minimal pause
    if available_tokens == 0 or total_remaining == 0 or earliest_reset is None:
        return 0.2  # Minimal pause when no rate limit info

    # Calculate time until reset
    time_until_reset = (earliest_reset - now_utc).total_seconds()
    if time_until_reset <= 0:
        return 0.1  # Very minimal pause if reset passed

    # Calculate optimal pause
    if estimated_requests > 0:
        # If we have plenty of remaining requests, use minimal pause
        if total_remaining > estimated_requests * 2:
            return 0.1  # We have plenty of capacity

        # Otherwise distribute requests over time
        safe_remaining = total_remaining * 0.9  # 10% buffer

        if estimated_requests >= safe_remaining:
            # More requests than capacity - need to be careful
            optimal_pause = time_until_reset / safe_remaining
        else:
            # Fewer requests than capacity - can go faster
            optimal_pause = time_until_reset / (total_remaining * 2)
    else:
        # No estimate - use very small pause
        optimal_pause = 0.2

    # Apply bounds: minimum 0.1s, maximum 5s
    optimal_pause = max(0.1, min(5.0, optimal_pause))

    # Add tiny random jitter
    jitter = np.random.uniform(-0.02, 0.02)
    optimal_pause += max(0, jitter)

    return round(optimal_pause, 2)

def estimate_request_count_fn(start_date, end_date, tasks_count):
    """
    Phase 4: Estimate number of API requests needed.

    For counts API: ~1 request per 30-31 days
    For search API: varies based on tweet volume
    """
    days_diff = (end_date - start_date).days + 1

    # Counts API: approximately 1 request per 30-31 days
    count_requests_per_task = math.ceil(days_diff / 30.5)
    total_count_requests = count_requests_per_task * tasks_count

    # Search API: учитываем актуальный max_results
    actual_max_results = 100 if 'context_annotations' in TWEET_FIELDS_COMPREHENSIVE else 500
    avg_search_requests_per_task = 1  # Will be updated based on actual counts

    return {
        'count_requests_per_task': count_requests_per_task,
        'total_count_requests': total_count_requests,
        'search_requests_estimate': avg_search_requests_per_task,
        'actual_max_results': actual_max_results
    }

def initialize_clients_fn():
    global clients_manager, KEYS_PATH, current_client_index_global, _current_active_token
    clients_manager.clear(); current_client_index_global = -1; _current_active_token = None
    token_files = ["bearer_token.txt", "bearer_token2.txt", "bearer_token3.txt"]
    loaded_tokens_count = 0
    if KEYS_PATH is None: update_status_display_fn(["Error: KEYS_PATH undefined."]); return False
    for i, token_file in enumerate(token_files):
        token_id = f"Token{i+1}"; full_token_path = os.path.join(KEYS_PATH, token_file); bearer_token_val = None
        try:
            if os.path.exists(full_token_path):
                with open(full_token_path, 'r') as f_token: bearer_token_val = f_token.read().strip()
                if bearer_token_val:
                    client_instance = tweepy.Client(bearer_token=bearer_token_val, wait_on_rate_limit=False)
                    clients_manager[token_id] = {
                        'client': client_instance,
                        'rate_limited_until': datetime.now(timezone.utc) - timedelta(seconds=1),
                        'bearer_token_value': bearer_token_val,
                        'last_used_timestamp': datetime.min.replace(tzinfo=timezone.utc),
                        'limits': {},
                        'request_count': 0  # Phase 4: Track requests per token
                    }
                    loaded_tokens_count +=1; update_status_display_fn([f"Initialized {token_id}."])
                else: update_status_display_fn([f"Warn: {token_file} empty."])
            else: update_status_display_fn([f"Warn: {token_file} not found at {full_token_path}."])
        except Exception as e_client_init: update_status_display_fn([f"Err init {token_id}: {str(e_client_init)[:50]}"])
    update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
    if loaded_tokens_count == 0: update_status_display_fn(["Critical: No Tokens loaded."]); return False
    update_status_display_fn([f"Initialized {loaded_tokens_count} client(s)."]); return True

def get_next_available_client_fn():
    """
    Возвращает:
        active_client – объект tweepy.Client, готовый к использованию;
        token_id_used – строковый идентификатор выбранного токена (например 'Token1').

    Логика (без рекурсии):
        1. Берём только токены, отмеченные галочками пользователем.
        2. Отбрасываем токены, находящиеся в состоянии rate-limitʼа.
        3. Считаем «score» (свободные лимиты + время простоя) и берём токен с
           максимальным score.
        4. Если все выбранные токены в rate-limitʼе – вычисляем ближайший момент
           сброса и ОДИН раз ждём нужное время, обновляя счётчик каждую секунду.
           После ожидания переходим к шагу 3 (всё внутри одного while-цикла).
    """
    global clients_manager, current_client_index_global, _current_active_token, token_selection_checkboxes

    # 1. Проверка, что клиенты вообще есть
    if not clients_manager:
        update_status_display_fn(["Fatal: No clients."])
        return None, None

    # 2. Определяем токены, отмеченные пользователем
    selected_tokens = [
        tid for tid in sorted(clients_manager.keys())
        if tid in token_selection_checkboxes and token_selection_checkboxes[tid].value
    ]
    if not selected_tokens:
        update_status_display_fn(["Error: No tokens selected for use!"])
        return None, None

    # 3. Основной бесконечный цикл (выход происходит при успешном выборе токена
    #    ИЛИ при непредвиденной ошибке/отсутствии данных)
    max_wait_cycles = 0  # Защита от бесконечного ожидания
    while max_wait_cycles < 10:  # Максимум 10 циклов ожидания
        best_token_id = None
        best_score    = -1
        now_utc       = datetime.now(timezone.utc)

        # 3а. Выбираем лучший доступный токен
        for token_id in selected_tokens:
            info = clients_manager[token_id]

            # Пропускаем токены, ещё находящиеся в блокировке
            if now_utc < info['rate_limited_until']:
                continue

            # Считаем «score»
            score = 0
            if 'limits' in info:
                limits = info['limits']
                score += limits.get('search_tweets', {}).get('remaining', 100) * 2
                score += limits.get('counts', {}).get('remaining', 100)
                score += limits.get('users', {}).get('remaining', 100) * 0.5
            else:
                score = 100  # нет данных о лимитах – даём базовый балл

            # Бонус за «отдых»
            idle_secs = (now_utc - info['last_used_timestamp']).total_seconds()
            score += min(50, idle_secs / 60)   # до +50 баллов

            if score > best_score:
                best_score   = score
                best_token_id = token_id

        # 3б. Если нашли подходящий токен – возвращаем его
        if best_token_id:
            _current_active_token = best_token_id
            chosen_info = clients_manager[best_token_id]
            chosen_info['last_used_timestamp'] = now_utc
            chosen_info['request_count'] += 1
            update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
            return chosen_info['client'], best_token_id

        # 3в. Если сюда дошли – ВСЕ выбранные токены всё ещё «красные».
        #     Считаем, сколько ждать до ближайшего сброса.
        future_resets = [
            clients_manager[tid]['rate_limited_until']
            for tid in selected_tokens
            if clients_manager[tid]['rate_limited_until'] > now_utc
        ]
        if not future_resets:
            # Не смогли определить время сброса (маловероятно) – выходим
            update_status_display_fn(["Emergency: Could not find available client."])
            return None, None

        earliest_reset = min(future_resets)
        wait_seconds   = max(1, int((earliest_reset - now_utc).total_seconds()))

        # Ограничиваем максимальное время ожидания
        if wait_seconds > 900:  # 15 минут максимум
            update_status_display_fn(["Error: All tokens rate-limited for >15 minutes. Check API status."])
            return None, None

        update_status_display_fn(
            [f"All tokens rate-limited. Waiting ~{wait_seconds} s until reset…"]
        )

        # 3г. «Грамотное ожидание» – показываем обратный отсчёт
        start = time.monotonic()
        while True:
            elapsed   = time.monotonic() - start
            remaining = wait_seconds - int(elapsed)
            if remaining <= 0:
                break
            # Обновляем каждые 5 секунд (или чаще, если осталось мало времени)
            update_status_display_fn([f"Waiting: {remaining} s…"], is_token_status_update=False)
            time.sleep(min(5, remaining))

        max_wait_cycles += 1
        # Цикл while True продолжится и попробует выбрать токен снова

    # Если достигли максимума циклов ожидания
    update_status_display_fn(["Error: Maximum wait cycles reached. Something is wrong with rate limits."])
    return None, None

def update_client_rate_limit_info_fn(token_id_val, headers_dict_val, endpoint_name_val="unknown"):
    """Update rate limit info from response headers."""
    global clients_manager
    if token_id_val not in clients_manager or not headers_dict_val:
        return

    try:
        # Extract rate limit headers
        limit = int(headers_dict_val.get('x-rate-limit-limit', 0))
        remaining = int(headers_dict_val.get('x-rate-limit-remaining', 0))
        reset_ts = int(headers_dict_val.get('x-rate-limit-reset', 0))

        # Determine endpoint type more accurately
        if 'count' in endpoint_name_val.lower() or 'get_all_tweets_count' in endpoint_name_val:
            endpoint_type = 'counts'
        elif 'search' in endpoint_name_val.lower() or 'search_all_tweets' in endpoint_name_val:
            endpoint_type = 'search_tweets'
        elif 'user' in endpoint_name_val.lower():
            endpoint_type = 'users'
        else:
            endpoint_type = 'unknown'

        # Store limit info
        if limit > 0:  # Only update if we got valid data
            clients_manager[token_id_val]['limits'][endpoint_type] = {
                'limit': limit,
                'remaining': remaining,
                'reset_ts': reset_ts,
                'checked_at_utc': datetime.now(timezone.utc).isoformat()
            }

        # Update rate_limited_until if exhausted
        if remaining == 0 and reset_ts > time.time():
            new_limit_until = datetime.fromtimestamp(reset_ts, timezone.utc) + timedelta(seconds=10)
            if new_limit_until > clients_manager[token_id_val]['rate_limited_until']:
                clients_manager[token_id_val]['rate_limited_until'] = new_limit_until
                update_status_display_fn([f"Rate limit hit: {token_id_val} for {endpoint_type}, reset at {new_limit_until:%H:%M:%S}."])

        # Update token status display
        update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)

    except Exception as e_hdr_parse:
        update_status_display_fn([f"Warn: Parse RL headers {token_id_val}: {e_hdr_parse}"])

# --- Field Definitions (Phase 2: Reduced Media/Poll fields) ---
TWEET_FIELDS_COMPREHENSIVE = [
    "created_at", "author_id", "conversation_id", "entities", "geo",
    "in_reply_to_user_id", "lang", "possibly_sensitive", "public_metrics",
    "referenced_tweets", "reply_settings", "source", "withheld",
    "attachments", "edit_history_tweet_ids",
    # Добавленные поля:
    "context_annotations",  # Семантические аннотации (важно для анализа!)
    "id",                   # ID твита (обычно по умолчанию, но лучше явно указать)
    "text"                  # Текст твита (обычно по умолчанию, но лучше явно указать)
]
EXPANSIONS_COMPREHENSIVE = ["author_id", "referenced_tweets.id", "in_reply_to_user_id", "attachments.media_keys", "attachments.poll_ids", "geo.place_id", "entities.mentions.username", "referenced_tweets.id.author_id"]
USER_FIELDS_COMPREHENSIVE = ["created_at", "description", "entities", "id", "location", "name", "pinned_tweet_id", "profile_image_url", "protected", "public_metrics", "url", "username", "verified", "verified_type", "withheld"]
MEDIA_FIELDS_COMPREHENSIVE = [
    "media_key", "type", "url", "preview_image_url", "duration_ms",
    "height", "width", "public_metrics", "alt_text",
    # Добавленные поля:
    "variants"  # Варианты для видео (разные битрейты/форматы)
]
POLL_FIELDS_COMPREHENSIVE = [
    "id", "options", "voting_status", "duration_minutes", "end_datetime"
    # Все основные поля уже есть - это полный набор для polls
]
PLACE_FIELDS_COMPREHENSIVE = ["contained_within", "country", "country_code", "full_name", "geo", "id", "name", "place_type"]

# --- Other Global Variables and Helper Functions ---
FIMI_EVENTS = ["1. The Black Sea Grain Initiative", "2. Finland and Sweden's NATO Entry", "3. Twin Earthquake", "4. Middle East", "5. Local and National Elections"]
ACTORS_POSTS_LOG_DIR = None
if 'LOGS_PATH' in globals() and LOGS_PATH is not None: ACTORS_POSTS_LOG_DIR = os.path.join(LOGS_PATH, "actors_posts_downloads"); os.makedirs(ACTORS_POSTS_LOG_DIR, exist_ok=True)
else: ACTORS_POSTS_LOG_DIR = "./local_logs/actors_posts_downloads"; os.makedirs(ACTORS_POSTS_LOG_DIR, exist_ok=True)
estimation_results_df = pd.DataFrame(); user_id_cache = {}; task_selection_checkboxes_global = []; user_confirmed_download_limit_global = 0
total_posts_found_across_all_counts = 0  # Счетчик общего количества найденных постов

def sanitize_filename(text_val, max_length=60):
    if not text_val or not text_val.strip(): return "NO_ADDITIONAL_QUERY"
    text_val = str(text_val); text_val = re.sub(r'[^\w\s-]', '', text_val); text_val = re.sub(r'[-\s]+', '_', text_val).strip('_'); return text_val[:max_length].rstrip('_')

def get_fimi_slug(fimi_event_name_str_val):
    parts = fimi_event_name_str_val.split('.', 1); num_part = parts[0].strip(); name_part_slug = sanitize_filename(parts[1].strip() if len(parts) > 1 else fimi_event_name_str_val.strip(), max_length=100); return f"{num_part}_{name_part_slug}"

def log_operation_fn(log_data_dict_val, stage_str_val="general"):
    global ACTORS_POSTS_LOG_DIR
    if not ACTORS_POSTS_LOG_DIR: update_status_display_fn(["Error: Log dir NA"]); return None
    try:
        ts_iso_utc = datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'); ts_filename_part = ts_iso_utc.replace(':','').replace('-','').replace('T','_').replace('Z','')
        script_name_part = "actors_posts_dl_script_ph4"; log_filename = f"{ts_filename_part}_{script_name_part}_{stage_str_val}_log.json"; log_filepath = os.path.join(ACTORS_POSTS_LOG_DIR, log_filename)
        with open(log_filepath, 'w', encoding='utf-8') as f_log: json.dump(log_data_dict_val, f_log, ensure_ascii=False, indent=2)
        return log_filepath
    except Exception as e_log: update_status_display_fn([f"LogWriteErr: {str(e_log)[:50]}"]); return None

def get_user_id_from_username_cached_fn(username_str_val):
    global user_id_cache, clients_manager
    clean_username = username_str_val.strip().lstrip('@')
    if not clean_username: return None
    if clean_username in user_id_cache: return user_id_cache[clean_username]

    active_client, token_id_used = get_next_available_client_fn()
    if not active_client: update_status_display_fn(["Err: No client for User ID lookup."]); return None

    try:
        # Phase 4: Dynamic pause for user lookup
        pause_duration = calculate_optimal_pause_fn('users', 1, clients_manager)
        time.sleep(pause_duration)

        user_response = active_client.get_users(usernames=[clean_username], user_fields=["id"])

        # Extract headers correctly
        resp_headers = {}
        if hasattr(user_response, '_headers'):
            resp_headers = dict(user_response._headers)
        elif hasattr(user_response, 'response') and hasattr(user_response.response, 'headers'):
            resp_headers = dict(user_response.response.headers)

        update_client_rate_limit_info_fn(token_id_used, resp_headers, f"get_users_for_{clean_username}")

        if user_response.data and len(user_response.data) > 0:
            user_id_val = user_response.data[0].id
            user_id_cache[clean_username] = user_id_val
            return user_id_val
        else:
            update_status_display_fn([f"Warn: User '{clean_username}' not found."])
            return None
    except tweepy.TooManyRequests as tmr_user_id:
        update_status_display_fn([f"RL on {token_id_used} UserID lookup for {clean_username}."])
        reset_time_unix = tmr_user_id.response.headers.get('x-rate-limit-reset')
        reset_time_dt = datetime.fromtimestamp(int(reset_time_unix), timezone.utc) if reset_time_unix else datetime.now(timezone.utc) + timedelta(minutes=16)
        clients_manager[token_id_used]['rate_limited_until'] = reset_time_dt + timedelta(seconds=10)
        update_client_rate_limit_info_fn(token_id_used, tmr_user_id.response.headers, f"get_users_tmr_for_{clean_username}")
        update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
        return get_user_id_from_username_cached_fn(username_str_val)
    except Exception as e_user_id:
        update_status_display_fn([f"Err get ID {clean_username}: {str(e_user_id)[:70]}"])
        return None

def check_user_activity_in_period(username, start_date, end_date):
    """
    Быстрая проверка, есть ли у пользователя посты в указанный период
    Returns: (has_posts, last_post_date, status_message)
    """
    active_client, token_id_used = get_next_available_client_fn()
    if not active_client:
        return None, None, "No client available"

    try:
        # Быстрый запрос - берем только 1 твит
        pause_duration = calculate_optimal_pause_fn('search_tweets', 1, clients_manager)
        time.sleep(pause_duration)

        # Запрос для получения последнего твита пользователя
        query = f"from:{username.lstrip('@')}"
        response = active_client.search_all_tweets(
            query=query,
            max_results=10,  # Берем 10 для надежности
            tweet_fields=["created_at"],
            sort_order="recency"  # Сортировка по новизне
        )

        # Обновляем rate limit info
        resp_headers = {}
        if hasattr(response, '_headers'):
            resp_headers = dict(response._headers)
        elif hasattr(response, 'response') and hasattr(response.response, 'headers'):
            resp_headers = dict(response.response.headers)

        update_client_rate_limit_info_fn(token_id_used, resp_headers, f"activity_check_{username[:10]}")

        if response.data and len(response.data) > 0:
            # Берем самый свежий твит
            latest_tweet = response.data[0]
            latest_date = datetime.fromisoformat(latest_tweet.created_at.replace('Z', '+00:00'))

            # Проверяем, попадает ли в период
            start_dt = datetime(start_date.year, start_date.month, start_date.day, tzinfo=timezone.utc)
            end_dt = datetime(end_date.year, end_date.month, end_date.day, 23, 59, 59, tzinfo=timezone.utc)

            if latest_date < start_dt:
                return False, latest_date, f"Last post before period ({latest_date.strftime('%Y-%m-%d')})"
            elif latest_date > end_dt:
                # Нужно проверить, есть ли посты В периоде
                # Делаем запрос с ограничением по датам
                time.sleep(pause_duration)
                period_response = active_client.search_all_tweets(
                    query=query,
                    start_time=start_dt.isoformat(),
                    end_time=end_dt.isoformat(),
                    max_results=10
                )

                if period_response.data and len(period_response.data) > 0:
                    return True, latest_date, f"Active in period (latest: {latest_date.strftime('%Y-%m-%d')})"
                else:
                    return False, latest_date, f"No posts in period (latest: {latest_date.strftime('%Y-%m-%d')})"
            else:
                return True, latest_date, f"Active in period (latest: {latest_date.strftime('%Y-%m-%d')})"
        else:
            return False, None, "No posts found"

    except Exception as e:
        return None, None, f"Error checking: {str(e)[:30]}"

def build_full_search_query_fn(username_str_val, tweet_type_api_filter_str, additional_query_text_str):
    """
    ИСПРАВЛЕНО: Добавлен return в конце функции
    Правильное построение поискового запроса для Twitter API v2
    Порядок: from:username (additional_query) filters
    """
    query_parts = []

    # 1. Добавляем username если есть
    if username_str_val and username_str_val.strip():
        query_parts.append(f"from:{username_str_val.lstrip('@')}")

    # 2. Добавляем дополнительный поисковый запрос
    if additional_query_text_str and additional_query_text_str.strip():
        clean_add_q = additional_query_text_str.strip()
        # Проверяем, нужно ли добавить скобки для группировки
        is_already_grouped = (clean_add_q.startswith("(") and clean_add_q.endswith(")"))
        if is_already_grouped:
            query_parts.append(clean_add_q)
        elif " OR " in clean_add_q.upper() or (" " in clean_add_q and not clean_add_q.startswith('"')):
            # Добавляем скобки если есть OR или пробелы (но не если это точная фраза в кавычках)
            query_parts.append(f"({clean_add_q})")
        else:
            query_parts.append(clean_add_q)

    # 3. Добавляем фильтры типа твита В КОНЕЦ
    if tweet_type_api_filter_str and tweet_type_api_filter_str.strip():
        # Убираем лишние скобки если они есть
        filter_str = tweet_type_api_filter_str.strip()
        if filter_str.startswith("(") and filter_str.endswith(")"):
            filter_str = filter_str[1:-1]
        query_parts.append(filter_str)

    final_query = " ".join(query_parts)

    return final_query

def build_batch_query_for_accounts(usernames_list, tweet_type_filter, additional_query, max_query_length=1024):
    """
    Строит запрос для нескольких аккаунтов сразу с учетом максимальной длины
    Returns: список запросов (каждый для группы аккаунтов)
    """
    if not usernames_list:
        return []

    # Базовая часть запроса
    base_parts = []
    if additional_query and additional_query.strip():
        clean_add_q = additional_query.strip()
        if " OR " in clean_add_q.upper() or (" " in clean_add_q and not clean_add_q.startswith('"')):
            base_parts.append(f"({clean_add_q})")
        else:
            base_parts.append(clean_add_q)

    if tweet_type_filter and tweet_type_filter.strip():
        filter_str = tweet_type_filter.strip()
        if filter_str.startswith("(") and filter_str.endswith(")"):
            filter_str = filter_str[1:-1]
        base_parts.append(filter_str)

    base_query = " ".join(base_parts) if base_parts else ""
    base_length = len(base_query) + 1 if base_query else 0  # +1 for space

    # Группируем аккаунты
    batches = []
    current_batch = []
    current_length = base_length

    for username in usernames_list:
        from_clause = f"from:{username.lstrip('@')}"
        # Длина с учетом OR между аккаунтами
        additional_length = len(from_clause) + (4 if current_batch else 0)  # " OR " = 4

        if current_length + additional_length > max_query_length - 10:  # -10 для безопасности
            if current_batch:
                batches.append(current_batch)
                current_batch = [username]
                current_length = base_length + len(from_clause)
            else:
                # Один аккаунт уже превышает лимит
                batches.append([username])
        else:
            current_batch.append(username)
            current_length += additional_length

    if current_batch:
        batches.append(current_batch)

    # Строим финальные запросы
    final_queries = []
    for batch in batches:
        if len(batch) == 1:
            query = build_full_search_query_fn(batch[0], tweet_type_filter, additional_query)
        else:
            # Объединяем через OR
            from_parts = [f"from:{u.lstrip('@')}" for u in batch]
            from_query = "(" + " OR ".join(from_parts) + ")"
            query_parts = [from_query]
            if base_query:
                query_parts.append(base_query)
            query = " ".join(query_parts)
        final_queries.append((query, batch))  # Возвращаем запрос и список аккаунтов в нем

    return final_queries

def check_existing_data_files_fn(fimi_slug_str_val, acc_name_str_val, query_tag_str_val, type_slug_str_val):
    global ACTORS_BASE_DIR
    if not ACTORS_BASE_DIR: return False
    data_dir = os.path.join(ACTORS_BASE_DIR, fimi_slug_str_val, acc_name_str_val, query_tag_str_val, type_slug_str_val)
    return os.path.isdir(data_dir) and any(f.endswith('.json') for f in os.listdir(data_dir))

def get_accurate_daily_total_count_fn(base_query_str_val, start_time_api_val_dt, end_time_api_val_dt, task_display_name_val="Task", estimated_requests=None):
    global clients_manager
    total_tweets_for_task = 0; current_pagination_token = None; days_processed_count = 0
    total_days_in_period = (end_time_api_val_dt.date() - start_time_api_val_dt.date()).days + 1
    requests_made = 0

    with tqdm(total=total_days_in_period, desc=f"Counts: {task_display_name_val[:20]}", unit="day", leave=False) as pbar_daily_counts:
        while True:
            active_client, token_id_used = get_next_available_client_fn()
            if not active_client:
                err_msg = f"FATAL: No client for daily count: {task_display_name_val}"
                update_status_display_fn([err_msg]); return -1, err_msg
            try:
                # Phase 4: Calculate optimal pause
                remaining_requests = estimated_requests - requests_made if estimated_requests else 10
                pause_duration = calculate_optimal_pause_fn('counts', remaining_requests, clients_manager)
                time.sleep(pause_duration)

                start_time_str = start_time_api_val_dt.isoformat(timespec='seconds').replace('+00:00', 'Z')
                end_time_str = end_time_api_val_dt.isoformat(timespec='seconds').replace('+00:00', 'Z')
                current_day_counts_response = active_client.get_all_tweets_count(
                    query=base_query_str_val, start_time=start_time_str, end_time=end_time_str,
                    granularity='day', next_token=current_pagination_token
                )
                requests_made += 1

                # Extract headers correctly
                resp_headers = {}
                if hasattr(current_day_counts_response, '_headers'):
                    resp_headers = dict(current_day_counts_response._headers)
                elif hasattr(current_day_counts_response, 'response') and hasattr(current_day_counts_response.response, 'headers'):
                    resp_headers = dict(current_day_counts_response.response.headers)

                update_client_rate_limit_info_fn(token_id_used, resp_headers, f"counts_{task_display_name_val[:15]}")

                days_in_this_page = 0
                if current_day_counts_response.data:
                    days_in_this_page = len(current_day_counts_response.data)
                    for day_data in current_day_counts_response.data: total_tweets_for_task += day_data.get('tweet_count', 0)

                pbar_daily_counts.update(days_in_this_page); days_processed_count += days_in_this_page
                pbar_daily_counts.set_postfix_str(f"{days_processed_count}/{total_days_in_period}d. Total Est: {total_tweets_for_task} | Pause: {pause_duration}s")

                current_pagination_token = current_day_counts_response.meta.get('next_token')
                if not current_pagination_token: break
            except tweepy.TooManyRequests as tmr_counts:
                update_status_display_fn([f"RL on {token_id_used} for daily counts: {task_display_name_val}."])
                reset_unix = tmr_counts.response.headers.get('x-rate-limit-reset')
                reset_dt = datetime.fromtimestamp(int(reset_unix), timezone.utc) if reset_unix else datetime.now(timezone.utc) + timedelta(minutes=16)
                clients_manager[token_id_used]['rate_limited_until'] = reset_dt + timedelta(seconds=10)
                update_client_rate_limit_info_fn(token_id_used, tmr_counts.response.headers, f"counts_tmr_{task_display_name_val[:10]}")
                update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
                pbar_daily_counts.set_postfix_str(f"RL on {token_id_used}, retrying counts page...")
                continue
            except Exception as e_counts:
                err_msg = f"Err daily cnt {task_display_name_val} w/{token_id_used}: {str(e_counts)[:30]}"
                update_status_display_fn([err_msg]); traceback.print_exc(); return -1, err_msg
    return total_tweets_for_task, "Estimated (Full Daily Pag.)"

clients_fully_initialized_flag = initialize_clients_fn()
token_selection_checkboxes = {}
if clients_manager:
    for token_id in sorted(clients_manager.keys()):
        token_selection_checkboxes[token_id] = widgets.Checkbox(
            value=True,
            description=f"Use {token_id}",
            indent=False
        )

token_selection_box = VBox(
    [widgets.HTML("<b>Select Active Tokens:</b>")] +
    list(token_selection_checkboxes.values()),
    layout=Layout(border='1px solid #ddd', padding='5px', margin='5px 0')
)

# --- UI Elements ---
style = {'description_width': 'initial'}
account_names_input = widgets.Textarea(value='XDevelopers,elonmusk', placeholder='Comma-separated Twitter usernames. Leave blank for global search if queries are provided.', description='Target Accounts:', layout=Layout(width='95%', height='50px'), style=style)
search_queries_input = widgets.Textarea(value='', placeholder='One query per line. Blank for all user posts if accounts provided.', description='Additional Search Queries (Optional):', layout=Layout(width='95%', height='80px'), style=style)
fimi_dropdown = widgets.Dropdown(options=FIMI_EVENTS, value=FIMI_EVENTS[0], description='Select FIMI Event Context:', layout=Layout(width='95%'), style=style)
default_start_date_val = py_date(2022, 1, 1); default_end_date_val = py_date(2024, 12, 31)
start_date_picker = widgets.DatePicker(description='Start Date:', value=default_start_date_val, style=style)
end_date_picker = widgets.DatePicker(description='End Date:', value=default_end_date_val, style=style)
date_box = HBox([start_date_picker, end_date_picker], layout=Layout(justify_content='space-around'))
tweet_type_options_desc_map = {
    "Original Posts (No RT/Reply/Quote)": "-is:retweet -is:reply -is:quote",  # Убрали скобки
    "Retweets (Native)": "is:retweet",
    "Quote Tweets (RT with Comment)": "is:quote",
    "Replies": "is:reply"
}
def create_proper_slug(desc, api_filter):
    """Создает правильный slug для типа твитов"""
    if "Original Posts" in desc:
        return "original_posts"
    else:
        return re.sub(r'[^\w_]', '', api_filter.replace(":", "_").replace("-", "").replace(" ", "_").lower())

tweet_type_slugs_map_dict = {
    desc: create_proper_slug(desc, api_filter)
    for desc, api_filter in tweet_type_options_desc_map.items()
}
tweet_type_checkboxes_map_ui = {desc: widgets.Checkbox(value=True, description=desc, indent=False, layout=Layout(width='auto')) for desc in tweet_type_options_desc_map.keys()}
select_all_tweet_types_cb_ui = widgets.Checkbox(value=True, description="Select/Deselect All Types", indent=False)
def on_select_all_tweet_types_changed_ui(change):
    for cb_item in tweet_type_checkboxes_map_ui.values(): cb_item.value = change.new
select_all_tweet_types_cb_ui.observe(on_select_all_tweet_types_changed_ui, names='value')

# Phase 4: Add optimization toggle
rate_limit_optimization_checkbox = widgets.Checkbox(value=True, description="Enable Smart Rate Limit Management (Phase 4)", indent=False)
batch_accounts_checkbox = widgets.Checkbox(
    value=True,
    description="Batch multiple accounts in count queries (faster estimation)",
    indent=False,
    style={'description_width': 'initial'}
)
check_activity_checkbox = widgets.Checkbox(
    value=False,  # По умолчанию выключено, т.к. требует дополнительные запросы
    description="Pre-check if accounts have posts in date range (uses extra API calls)",
    indent=False,
    style={'description_width': 'initial'}
)
def on_rate_limit_optimization_changed(change):
    global rate_limit_optimization_enabled
    rate_limit_optimization_enabled = change.new
    update_status_display_fn([f"Rate limit optimization {'enabled' if change.new else 'disabled'}"])
rate_limit_optimization_checkbox.observe(on_rate_limit_optimization_changed, names='value')

tweet_types_box_layout_ui = VBox([select_all_tweet_types_cb_ui] + list(tweet_type_checkboxes_map_ui.values()) + [widgets.HTML("<hr>"), rate_limit_optimization_checkbox, batch_accounts_checkbox, check_activity_checkbox], layout=Layout(border='1px solid #ccc', padding='10px', margin='5px 0'))
estimation_output_area_ui = widgets.Output(layout=Layout(border='1px solid #eee', padding='10px', margin='10px 0', width='95%', overflow_x='auto'))
download_confirmation_dialog_output_ui = widgets.Output(layout=Layout(margin='10px 0', width='95%'))
estimate_button_ui = widgets.Button(description="1. Estimate Tweet Counts", icon='search', button_style='primary', layout=Layout(width='auto'))
download_button_ui = widgets.Button(description="2. Download Selected Tweets", icon='download', button_style='success', disabled=True, layout=Layout(width='auto'))
reset_button_ui = widgets.Button(description="Reset Form", icon='refresh', button_style='warning', layout=Layout(width='auto'))
action_buttons_box_ui = HBox([estimate_button_ui, download_button_ui, reset_button_ui], layout=Layout(justify_content='space-around', margin='10px 0'))
user_limit_input_widget_ui = widgets.IntText(value=1000, description='Max Tweets to Download (Total for this Run):', style=style, layout=Layout(width='auto'))
confirm_limit_button_widget_ui = widgets.Button(description="Confirm Limit & Start Download", button_style='danger', layout=Layout(width='auto'))

@estimate_button_ui.on_click
def handle_estimate_button_click_fn(b_est):
    """
    FIX 29-May-2025
    ----------------
    • Правильно считаем количество задач в batch-режиме: теперь одна
      «задача» = 1 батч (query × type × batch), а не (accounts × …).
    • Прогресс-бар, оценка числа API-запросов и логика пауз теперь
      ориентируются на реальное число батч-запросов (в вашем примере
      4 query × 4 type = 16, а не 64).
    • В сам запрос к API и сохранение результатов никакие изменения
      не внесены — функционал остался тем же.
    """

    # ---------- базовые проверки ----------
    global estimation_results_df, download_button_ui, task_selection_checkboxes_global
    global clients_manager, clients_fully_initialized_flag, estimated_requests_per_task

    if not clients_fully_initialized_flag or not clients_manager:
        update_status_display_fn(["Error: Clients not initialized."])
        return

    download_button_ui.disabled = True
    estimation_output_area_ui.clear_output()
    download_confirmation_dialog_output_ui.clear_output()
    task_selection_checkboxes_global.clear()
    estimated_requests_per_task.clear()

    update_status_display_fn(
        ["Starting ACCURATE tweet count estimation (fixed batch logic)…"]
    )

    # ---------- читаем параметры из UI ----------
    acc_names_raw_val       = account_names_input.value
    queries_raw_list_val    = [q.strip() for q in search_queries_input.value.split('\n')]
    if not any(queries_raw_list_val):
        queries_raw_list_val = [""]                       # «пустой» запрос

    fimi_event_sel          = fimi_dropdown.value
    fimi_path_slug          = get_fimi_slug(fimi_event_sel)

    start_date_val_dt       = start_date_picker.value
    end_date_val_dt         = end_date_picker.value
    start_time_dt_api_val   = datetime(
        start_date_val_dt.year, start_date_val_dt.month, start_date_val_dt.day,
        0, 0, 0, tzinfo=timezone.utc
    )
    end_time_dt_api_val     = datetime(
        end_date_val_dt.year,   end_date_val_dt.month,   end_date_val_dt.day,
        23, 59, 59, tzinfo=timezone.utc
    )

    selected_types_desc_list = [
        desc for desc, cb_val in tweet_type_checkboxes_map_ui.items() if cb_val.value
    ]

    # ---------- разбираем аккаунты / режимы ----------
    usernames_to_proc_list = []
    is_global_search = False

    if acc_names_raw_val.strip():
        usernames_to_proc_list = [
            name.strip() for name in acc_names_raw_val.split(',') if name.strip()
        ]
    elif queries_raw_list_val != [""]:
        # глобальный поиск без конкретных аккаунтов
        is_global_search = True
        usernames_to_proc_list = ["GLOBAL_SEARCH"]
    else:
        update_status_display_fn(
            ["Error: Accounts & Queries can't both be empty."]
        )
        return

    if not selected_types_desc_list and not is_global_search:
        update_status_display_fn(
            ["Error: At least one tweet type required for user-specific searches."]
        )
        return
    if is_global_search and not any(q.strip() for q in queries_raw_list_val):
        update_status_display_fn(
            ["Error: Search queries cannot be empty for a global search."]
        )
        return

    # ---------- определяем, будет ли batch-режим ----------
    batch_mode = (
        batch_accounts_checkbox.value
        and not is_global_search
        and len(usernames_to_proc_list) > 1
    )

    # ---------- считаем точное число «задач» ----------
    if batch_mode:
        total_est_combos = 0
        for q in queries_raw_list_val:
            for t in selected_types_desc_list:
                # число батчей для данной пары (q, t)
                batches_cnt = len(
                    build_batch_query_for_accounts(
                        usernames_to_proc_list,
                        tweet_type_options_desc_map.get(t, ""),
                        q
                    )
                )
                total_est_combos += batches_cnt
    else:
        num_type_iterations = len(selected_types_desc_list) if not is_global_search else 1
        total_est_combos = (
            len(usernames_to_proc_list) * len(queries_raw_list_val) * num_type_iterations
        )

    # ---------- оценка числа API-запросов ----------
    request_estimates = estimate_request_count_fn(
        start_date_val_dt, end_date_val_dt, total_est_combos
    )
    update_status_display_fn(
        [f"Phase 4: Estimated ≈ {request_estimates['total_count_requests']} "
         f"count-API requests (with batching = {batch_mode})."]
    )

    # ---------- логируем старт ----------
    log_est_start_data = {
        "stage": "est_start_accurate_ph4",
        "ts_utc": datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'),
        "params": {
            "acc_mode": "GLOBAL" if is_global_search else usernames_to_proc_list,
            "queries": queries_raw_list_val,
            "fimi": fimi_event_sel,
            "start_dt": start_time_dt_api_val.isoformat(),
            "end_dt": end_time_dt_api_val.isoformat(),
            "types": selected_types_desc_list if not is_global_search else "N/A_Global",
            "batch_mode": batch_mode
        },
        "request_estimates": request_estimates
    }
    log_operation_fn(log_est_start_data, "est_start_accurate_ph4")

    # ---------- подготовка к циклу ----------
    estimated_tasks_list = []
    global total_posts_found_across_all_counts
    total_posts_found_across_all_counts = 0

    # основной прогресс-бар
    with tqdm(total=total_est_combos,
              desc="Overall Accurate Estimation (Ph4)") as pbar_overall_est_ui:

        # ====== 1. BATCH-режим =================================================
        if batch_mode:
            update_status_display_fn(
                ["Batching accounts for faster count estimation (fixed)…"]
            )

            for query_text_iter_val in queries_raw_list_val:
                query_tag_for_file_path = sanitize_filename(query_text_iter_val)

                for type_desc_iter_val in selected_types_desc_list:
                    type_api_filter_str = tweet_type_options_desc_map.get(
                        type_desc_iter_val, ""
                    )
                    type_slug_for_file_path = tweet_type_slugs_map_dict.get(
                        type_desc_iter_val, "all_types"
                    )

                    # строим батч-запросы
                    batched_queries = build_batch_query_for_accounts(
                        usernames_to_proc_list,
                        type_api_filter_str,
                        query_text_iter_val
                    )

                    for batch_num, (batch_query, accounts_in_batch) in enumerate(batched_queries, start=1):
                        task_display_name = (
                            f"Batch{batch_num}_{len(accounts_in_batch)}acc_"
                            f"{query_tag_for_file_path[:10]}_"
                            f"{type_desc_iter_val[:10]}"
                        )
                        pbar_overall_est_ui.set_postfix_str(task_display_name)

                        # === вызов counts API на батч ===
                        task_estimated_requests = request_estimates['count_requests_per_task']
                        batch_count_val, count_status_msg_val = (
                            get_accurate_daily_total_count_fn(
                                batch_query,
                                start_time_dt_api_val,
                                end_time_dt_api_val,
                                task_display_name,
                                task_estimated_requests,
                            )
                        )

                        # суммарно для трекера
                        if batch_count_val > 0:
                            total_posts_found_across_all_counts += batch_count_val
                            pbar_overall_est_ui.set_description(
                                f"Overall Est. (Ph4) – Found: "
                                f"{total_posts_found_across_all_counts:,}"
                            )

                        # ---------- добавляем ОДНУ строку на батч ----------
                        estimated_tasks_list.append({
                            'Account': ", ".join(accounts_in_batch),
                            'Accounts': ", ".join(accounts_in_batch),
                            'Accounts_List': accounts_in_batch,          # пригодится при скачивании
                            'FIMI_Event': fimi_event_sel,
                            'Search_Query_Input': query_text_iter_val if query_text_iter_val else "N/A",
                            'Query_Tag_For_Path': query_tag_for_file_path,
                            'Tweet_Type_Desc': type_desc_iter_val,
                            'Tweet_Type_Filter_API': type_api_filter_str or "N/A",
                            'Tweet_Type_Slug_For_Path': type_slug_for_file_path,
                            'Estimated_Count': batch_count_val,
                            'Actual_API_Query_Used': batch_query,        # именно групповой запрос
                            'Start_Time_API': start_time_dt_api_val.isoformat(timespec='seconds').replace('+00:00', 'Z'),
                            'End_Time_API': end_time_dt_api_val.isoformat(timespec='seconds').replace('+00:00', 'Z'),
                            'Status': count_status_msg_val,
                            'Data_Exists_Hint': False,                   # для группового запроса не проверяем
                            'Task_Display_Name': task_display_name,
                        })

                        # ←----------- прогресс-бар – теперь +1 (один батч)
                        pbar_overall_est_ui.update(1)

        # ====== 2. Обычный режим (по одному аккаунту) =========================
        else:
            # (этот блок не менялся)
            # ------------------------------------------------------------------
            for username_iter_val in usernames_to_proc_list:
                user_id_for_logging = None
                actual_username_for_query = None
                display_account_name = username_iter_val

                if username_iter_val == "GLOBAL_SEARCH":
                    actual_username_for_query = None
                else:
                    actual_username_for_query = username_iter_val
                    user_id_for_logging = get_user_id_from_username_cached_fn(
                        actual_username_for_query
                    )

                for query_text_iter_val in queries_raw_list_val:
                    query_tag_for_file_path = sanitize_filename(query_text_iter_val)

                    types_to_iterate_list = (
                        selected_types_desc_list
                        if not is_global_search
                        else ["N/A (Global Query Default)"]
                    )
                    for type_desc_iter_val in types_to_iterate_list:
                        task_display_name_short = (
                            f"{display_account_name[:10]}_"
                            f"{query_tag_for_file_path[:10]}_"
                            f"{type_desc_iter_val[:10]}"
                        )
                        pbar_overall_est_ui.set_postfix_str(task_display_name_short)

                        type_api_filter_str = (
                            tweet_type_options_desc_map.get(type_desc_iter_val, "")
                            if type_desc_iter_val != "N/A (Global Query Default)"
                            else ""
                        )
                        type_slug_for_file_path = (
                            tweet_type_slugs_map_dict.get(type_desc_iter_val, "all_types")
                            if type_desc_iter_val != "N/A (Global Query Default)"
                            else "all_types_global"
                        )

                        full_api_query_built_str = build_full_search_query_fn(
                            actual_username_for_query,
                            type_api_filter_str,
                            query_text_iter_val,
                        )
                        if not full_api_query_built_str:
                            update_status_display_fn(
                                [f"Skipping invalid empty query for \"{task_display_name_short}\""]
                            )
                            pbar_overall_est_ui.update(1)
                            continue

                        task_estimated_requests = request_estimates['count_requests_per_task']
                        current_estimated_count_val, count_status_msg_val = (
                            get_accurate_daily_total_count_fn(
                                full_api_query_built_str,
                                start_time_dt_api_val,
                                end_time_dt_api_val,
                                task_display_name_short,
                                task_estimated_requests,
                            )
                        )

                        if current_estimated_count_val > 0:
                            total_posts_found_across_all_counts += current_estimated_count_val
                            pbar_overall_est_ui.set_description(
                                f"Overall Est. (Ph4) – Found: "
                                f"{total_posts_found_across_all_counts:,}"
                            )

                            actual_max_results = request_estimates.get('actual_max_results', 100)
                            estimated_search_requests = math.ceil(
                                current_estimated_count_val / actual_max_results
                            )
                            estimated_requests_per_task[task_display_name_short] = (
                                estimated_search_requests
                            )

                        data_exists_hint_flag = check_existing_data_files_fn(
                            fimi_path_slug,
                            display_account_name,
                            query_tag_for_file_path,
                            type_slug_for_file_path,
                        )
                        if (data_exists_hint_flag and
                                "Error" not in count_status_msg_val and
                                "Fatal" not in count_status_msg_val):
                            count_status_msg_val += " (Exists?)"

                        estimated_tasks_list.append({
                            'Account': display_account_name,
                            'User_ID_Ref': user_id_for_logging,
                            'FIMI_Event': fimi_event_sel,
                            'Search_Query_Input': (
                                query_text_iter_val if query_text_iter_val else "N/A"
                            ),
                            'Query_Tag_For_Path': query_tag_for_file_path,
                            'Tweet_Type_Desc': type_desc_iter_val,
                            'Tweet_Type_Filter_API': type_api_filter_str or "N/A",
                            'Tweet_Type_Slug_For_Path': type_slug_for_file_path,
                            'Estimated_Count': current_estimated_count_val,
                            'Actual_API_Query_Used': full_api_query_built_str,
                            'Start_Time_API': start_time_dt_api_val.isoformat(timespec='seconds').replace('+00:00', 'Z'),
                            'End_Time_API': end_time_dt_api_val.isoformat(timespec='seconds').replace('+00:00', 'Z'),
                            'Status': count_status_msg_val,
                            'Data_Exists_Hint': data_exists_hint_flag,
                            'Task_Display_Name': task_display_name_short,
                        })

                        pbar_overall_est_ui.update(1)

    # ---------- формируем DataFrame результатов ----------
    estimation_results_df = pd.DataFrame(estimated_tasks_list)

    # Phase 4: Log token usage summary
    token_usage_summary = {token_id: info['request_count'] for token_id, info in clients_manager.items()}

    log_est_results_data = {"stage": "est_results_display_accurate_ph4", "ts_utc": datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'), "params_used": log_est_start_data["params"], "results_df": estimation_results_df.to_dict(orient='records') if not estimation_results_df.empty else "No results", "token_usage": token_usage_summary}
    log_operation_fn(log_est_results_data, "est_results_display_accurate_ph4")

    with estimation_output_area_ui:
        clear_output(wait=True)

        # ⇢ ИСПРАВЛЕНИЕ: убрано дублирование преобразования Estimated_Count
        # Теперь делаем это только один раз и правильно обрабатываем отрицательные числа
        if not estimation_results_df.empty:
            # Преобразуем Estimated_Count в число, учитывая возможные отрицательные значения
            def safe_extract_number(value):
                """Безопасно извлекает число из значения"""
                if isinstance(value, (int, float)):
                    return int(value)
                str_val = str(value)
                # Ищем числа, включая отрицательные
                import re
                match = re.search(r'-?\d+', str_val)
                if match:
                    return int(match.group())
                return 0

            estimation_results_df['Estimated_Count'] = estimation_results_df['Estimated_Count'].apply(safe_extract_number)

            # Пересортируем по обновлённым числам
            estimation_results_df = estimation_results_df.sort_values(
                by='Estimated_Count', ascending=False
            )

        # ⇢ 2. Отрисовываем таблицу
        if not estimation_results_df.empty:
            display(IPHTML(
                "<h4>Accurate Estimation Results (Phase 4 – Optimized):</h4>"
                "<p>Review counts and select tasks for download.</p>"
            ))

            header_lbls = [
                "Select", "Account", "Add. Query",
                "Tweet Type", "Est. Count", "Status"
            ]
            col_widths = ['7%', '15%', '25%', '20%', '10%', '23%']

            header_row = HBox(
                [
                    widgets.Label(
                        header_lbls[i],
                        layout=Layout(width=col_widths[i], font_weight='bold')
                    )
                    for i in range(len(header_lbls))
                ]
            )

            ui_rows = [header_row]
            task_selection_checkboxes_global.clear()        # сбрасываем старые чекбоксы

            for df_idx, task_row in estimation_results_df.iterrows():
                # число твитов для логики «выбрать/отключить»
                count_val = task_row['Estimated_Count']
                err_flag  = ("Error" in str(task_row['Status'])) or ("Fatal" in str(task_row['Status']))
                is_task_enabled = (count_val > 0) and (not err_flag)

                cb = widgets.Checkbox(
                    value=is_task_enabled,
                    description="",
                    indent=False,
                    disabled=not is_task_enabled,
                    layout=Layout(width=col_widths[0])
                )
                task_selection_checkboxes_global.append(
                    {'checkbox': cb, 'task_df_index': df_idx}
                )

                row_vals = [
                    task_row['Account'],
                    task_row['Search_Query_Input'],
                    task_row['Tweet_Type_Desc'],
                    task_row['Estimated_Count'],
                    task_row['Status']
                ]

                row_widgets = [cb] + [
                    widgets.Label(
                        (str(v)[:35] + "…") if len(str(v)) > 35 else str(v),
                        layout=Layout(width=col_widths[i + 1])
                    )
                    for i, v in enumerate(row_vals)
                ]
                ui_rows.append(HBox(row_widgets))

            display(VBox(ui_rows))

            # ⇢ 3. Общее количество возможных твитов
            # ИСПРАВЛЕНИЕ: более надёжный подсчёт
            total_sel_tweets = 0
            for _, row in estimation_results_df.iterrows():
                count = row['Estimated_Count']
                status = str(row['Status'])
                if count > 0 and not any(err in status.lower() for err in ['error', 'fatal']):
                    total_sel_tweets += count

            display(IPHTML(
                f"<p style='margin-top:10px;'>"
                f"<b>Total potentially downloadable tweets (accurate): "
                f"{total_sel_tweets}</b></p>"
            ))
            display(IPHTML(
                "<p><i>Phase 4: Token usage during estimation – "
                f"{', '.join([f'{k}: {v} requests' for k, v in token_usage_summary.items()])}"
                "</i></p>"
            ))

            # ИСПРАВЛЕНИЕ: добавляем отладочную информацию
            if total_sel_tweets == 0:
                display(IPHTML(
                    "<p style='color:orange;'><b>Debug info:</b> No downloadable tweets found. "
                    f"DataFrame has {len(estimation_results_df)} rows. "
                    f"Rows with count > 0: {len(estimation_results_df[estimation_results_df['Estimated_Count'] > 0])}. "
                    f"Rows without errors: {len(estimation_results_df[~estimation_results_df['Status'].str.contains('Error|Fatal', case=False, na=False)])}.</p>"
                ))

            # Разблокируем кнопку скачивания, если есть что скачивать
            download_button_ui.disabled = (total_sel_tweets == 0)
            if total_sel_tweets == 0:
                display(IPHTML(
                    "<p><i>No tweets available for download based on accurate estimations.</i></p>"
                ))
            else:
                display(IPHTML(
                    "<p style='color:green;'><b>✓ Download button is now enabled!</b></p>"
                ))

        else:
            display(IPHTML("<p>No accurate estimation data to display.</p>"))
            download_button_ui.disabled = True

    update_status_display_fn(["Phase 4 estimation complete with optimized rate limiting."])

@download_button_ui.on_click
def handle_download_button_click_fn(b_dl_ui):
    global estimation_results_df, task_selection_checkboxes_global, user_confirmed_download_limit_global
    selected_df_indices_vals = [item['task_df_index'] for item in task_selection_checkboxes_global if item['checkbox'].value and not item['checkbox'].disabled]
    with download_confirmation_dialog_output_ui:
        clear_output(wait=True)
        if not selected_df_indices_vals: display(IPHTML("<p style='color:orange;'>No tasks selected for download.</p>")); return
        sel_tasks_to_conf_df = estimation_results_df.loc[selected_df_indices_vals].copy()
        dlable_tasks_df = sel_tasks_to_conf_df[(sel_tasks_to_conf_df['Estimated_Count'] > 0) & (~sel_tasks_to_conf_df['Status'].str.contains("Err|Fatal", case=False, na=False))].copy()
        if dlable_tasks_df.empty: display(IPHTML("<p style='color:orange;'>Selected tasks not eligible for download.</p>")); return
        total_est_for_sel = dlable_tasks_df['Estimated_Count'].sum()
        display(IPHTML(f"<h4>Confirm Download Limit</h4><p>Selected tasks total: <b>{total_est_for_sel}</b> potential tweets (Accurate Estimate).</p><p>Monthly pull limit: 1,000,000. Enter max for THIS RUN:</p>"))
        user_limit_input_widget_ui.value = min(total_est_for_sel, 200000); user_limit_input_widget_ui.max = total_est_for_sel if total_est_for_sel > 0 else 1
        try: confirm_limit_button_widget_ui.on_click(on_confirm_limit_button_widget_clicked_fn, remove=True)
        except: pass
        confirm_limit_button_widget_ui.on_click(on_confirm_limit_button_widget_clicked_fn)
        display(HBox([user_limit_input_widget_ui, confirm_limit_button_widget_ui]))

def on_confirm_limit_button_widget_clicked_fn(b_conf_limit_ui):
    global user_confirmed_download_limit_global
    user_confirmed_download_limit_global = user_limit_input_widget_ui.value
    with download_confirmation_dialog_output_ui:
        clear_output(wait=True)
        if user_confirmed_download_limit_global <= 0: display(IPHTML("<p style='color:red;'>Download cancelled or zero limit.</p>")); update_status_display_fn(["Download aborted."]); return
        display(IPHTML(f"<p style='color:green;'>Download limit: <b>{user_confirmed_download_limit_global}</b>. Starting Phase 4 optimized download...</p>"))
    trigger_actual_download_process_phase4_fn()

def trigger_actual_download_process_phase4_fn():
    global estimation_results_df, user_confirmed_download_limit_global, ACTORS_BASE_DIR, task_selection_checkboxes_global, clients_manager, clients_fully_initialized_flag, estimated_requests_per_task
    if not clients_fully_initialized_flag or not clients_manager: update_status_display_fn(["Error: No clients for download."]); return
    if user_confirmed_download_limit_global <= 0: update_status_display_fn(["DL limit zero/negative."]); return
    if not ACTORS_BASE_DIR: update_status_display_fn(["Error: ACTORS_BASE_DIR not set."]); return

    update_status_display_fn([f"Init Phase 4 DL with fixed pause. Global limit: {user_confirmed_download_limit_global}."])
    selected_df_indices = [item['task_df_index'] for item in task_selection_checkboxes_global if item['checkbox'].value and not item['checkbox'].disabled]
    if not selected_df_indices: update_status_display_fn(["No tasks selected for DL."]); return

    tasks_df_selected = estimation_results_df.loc[selected_df_indices].copy()
    tasks_to_process_orig = tasks_df_selected[(tasks_df_selected['Estimated_Count'] > 0) & (~tasks_df_selected['Status'].str.contains("Err|Fatal", case=False, na=False))].sort_values(by='Estimated_Count', ascending=False).copy()
    if tasks_to_process_orig.empty: update_status_display_fn(["No selected tasks eligible for DL."]); return

    total_tweets_downloaded_run = 0; download_log_list = []

    # Определяем current_status и update_download_status ОДИН РАЗ в начале функции
    current_status = {
        'accounts_processed': 0,
        'total_accounts': 0,
        'current_account': '',
        'tweets_downloaded': 0,
        'total_target': user_confirmed_download_limit_global,
        'current_type': ''
    }

    def update_download_status():
        status_msg = f"[{current_status['accounts_processed']}/{current_status['total_accounts']}] {current_status['current_account']}: {current_status['tweets_downloaded']} tweets"
        if current_status['current_type']:
            status_msg += f" ({current_status['current_type']})"
        update_status_display_fn([status_msg])

    # Подробное логирование
    detailed_log = {
        'session_start': datetime.now(timezone.utc).isoformat(),
        'accounts_processing_log': [],
        'api_calls_log': [],
        'rate_limit_events': [],
        'errors_log': []
    }

    def log_detailed_event(event_type, details):
        detailed_log[f'{event_type}_log'].append({
            'timestamp': datetime.now(timezone.utc).isoformat(),
            'details': details
        })

        # Сохраняем лог каждые 10 событий
        if len(detailed_log.get(f'{event_type}_log', [])) % 10 == 0:
            log_file_path = log_operation_fn(detailed_log, f"detailed_download_{event_type}")
            if log_file_path:
                print(f"Detailed log saved: {log_file_path}")

    log_dl_start_data = {"stage": "dl_process_start_ph4", "ts_utc": datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'), "user_conf_limit": user_confirmed_download_limit_global, "tasks_summary": tasks_to_process_orig[['Account', 'Search_Query_Input', 'Tweet_Type_Desc', 'Estimated_Count']].to_dict(orient='records'), "rate_limit_optimization": "disabled_fixed_1.1s"}
    log_operation_fn(log_dl_start_data, "dl_process_start_ph4")

    if 'context_annotations' in TWEET_FIELDS_COMPREHENSIVE:
        api_page_max_results = 100
    else:
        api_page_max_results = 500

    # Фиксированная пауза
    FIXED_PAUSE_SECONDS = 1.1

    # Phase 4: Reset request counters
    for token_id in clients_manager:
        clients_manager[token_id]['request_count'] = 0

    # Считаем общее количество твитов для скачивания
    total_tweets_to_download = min(
        tasks_to_process_orig['Estimated_Count'].sum(),
        user_confirmed_download_limit_global
    )

    # Добавляем структуру для сбора статистики
    download_statistics = {}  # {account: {tweet_type: count}}

    with tqdm(total=total_tweets_to_download, desc="Overall Download Progress (Ph4)", unit="tweets") as pbar_overall_dl_tasks:
        pbar_overall_dl_tasks.set_postfix_str(f"0/{total_tweets_to_download} tweets")
        for task_idx, current_task_series_data in tasks_to_process_orig.iterrows():
            if total_tweets_downloaded_run >= user_confirmed_download_limit_global:
                update_status_display_fn(["Global DL limit for this run reached."]); break

            task_short_name = current_task_series_data.get('Task_Display_Name', f"{current_task_series_data['Account'][:10]}_{current_task_series_data['Query_Tag_For_Path'][:10]}_{current_task_series_data['Tweet_Type_Slug_For_Path'][:10]}")
            pbar_overall_dl_tasks.set_postfix_str(task_short_name)

            # Проверяем, это batch-строка или обычная
            is_batch = 'Accounts_List' in current_task_series_data and current_task_series_data['Accounts_List']

            if is_batch:
                # Batch режим - обрабатываем каждый аккаунт отдельно
                accounts_in_batch = current_task_series_data['Accounts_List']
                total_tweets_for_batch = 0

                # Обновляем общее количество аккаунтов для batch
                current_status['total_accounts'] = len(accounts_in_batch)
                current_status['accounts_processed'] = 0
                current_status['current_type'] = current_task_series_data['Tweet_Type_Desc']

                for acc_index, account in enumerate(accounts_in_batch, 1):
                    if total_tweets_downloaded_run >= user_confirmed_download_limit_global:
                        break

                    # Обновляем статус
                    current_status['current_account'] = account
                    current_status['accounts_processed'] = acc_index
                    current_status['tweets_downloaded'] = total_tweets_downloaded_run
                    update_download_status()

                    # Логируем начало обработки аккаунта
                    log_detailed_event('accounts_processing', {
                        'account': account,
                        'position': f"{acc_index}/{len(accounts_in_batch)}",
                        'tweet_type': current_task_series_data['Tweet_Type_Desc'],
                        'tweets_so_far': total_tweets_downloaded_run
                    })

                    # Создаем отдельный запрос для каждого аккаунта
                    single_account_query = build_full_search_query_fn(
                        account,
                        current_task_series_data['Tweet_Type_Filter_API'],
                        current_task_series_data['Search_Query_Input']
                    )

                    # Увеличиваем лимит на аккаунт
                    account_limit = min(5000, user_confirmed_download_limit_global - total_tweets_downloaded_run)
                    tweets_for_account = 0

                    # Стандартная логика скачивания для одного аккаунта
                    active_dl_client_inst, token_id_dl_used_str = get_next_available_client_fn()
                    if not active_dl_client_inst:
                        update_status_display_fn([f"No client available for {account}, skipping"])
                        continue

                    try:
                        time.sleep(FIXED_PAUSE_SECONDS)

                        # Подготавливаем путь, но НЕ создаем директорию сразу
                        fimi_slug_path = get_fimi_slug(current_task_series_data['FIMI_Event'])
                        save_directory_path = os.path.join(
                            ACTORS_BASE_DIR,
                            fimi_slug_path,
                            account.lstrip('@'),
                            current_task_series_data['Query_Tag_For_Path'],
                            current_task_series_data['Tweet_Type_Slug_For_Path']
                        )
                        # Флаг для отслеживания, создана ли директория
                        directory_created = False

                        # Пагинация для аккаунта
                        tweet_paginator_instance = tweepy.Paginator(
                            active_dl_client_inst.search_all_tweets,
                            query=single_account_query,
                            start_time=current_task_series_data['Start_Time_API'],
                            end_time=current_task_series_data['End_Time_API'],
                            max_results=api_page_max_results,
                            tweet_fields=TWEET_FIELDS_COMPREHENSIVE,
                            expansions=EXPANSIONS_COMPREHENSIVE,
                            user_fields=USER_FIELDS_COMPREHENSIVE,
                            media_fields=MEDIA_FIELDS_COMPREHENSIVE,
                            poll_fields=POLL_FIELDS_COMPREHENSIVE,
                            place_fields=PLACE_FIELDS_COMPREHENSIVE
                        )

                        pages_fetched = 0
                        for page_response_obj in tweet_paginator_instance:
                            if tweets_for_account >= account_limit or total_tweets_downloaded_run >= user_confirmed_download_limit_global:
                                break

                            pages_fetched += 1
                            # Extract headers
                            page_headers = {}
                            if hasattr(page_response_obj, '_headers'):
                                page_headers = dict(page_response_obj._headers)
                            elif hasattr(page_response_obj, 'response') and hasattr(page_response_obj.response, 'headers'):
                                page_headers = dict(page_response_obj.response.headers)

                            update_client_rate_limit_info_fn(token_id_dl_used_str, page_headers, f"dl_batch_split_{account[:10]}")

                            # Convert includes to serializable format
                            serializable_includes = {}
                            if hasattr(page_response_obj, 'includes') and page_response_obj.includes:
                                if hasattr(page_response_obj.includes, 'users') and page_response_obj.includes.users:
                                    serializable_includes['users'] = [user.data for user in page_response_obj.includes.users]
                                if hasattr(page_response_obj.includes, 'media') and page_response_obj.includes.media:
                                    serializable_includes['media'] = [media.data for media in page_response_obj.includes.media]
                                if hasattr(page_response_obj.includes, 'polls') and page_response_obj.includes.polls:
                                    serializable_includes['polls'] = [poll.data for poll in page_response_obj.includes.polls]
                                if hasattr(page_response_obj.includes, 'places') and page_response_obj.includes.places:
                                    serializable_includes['places'] = [place.data for place in page_response_obj.includes.places]
                                if hasattr(page_response_obj.includes, 'tweets') and page_response_obj.includes.tweets:
                                    serializable_includes['tweets'] = [tweet.data for tweet in page_response_obj.includes.tweets]

                            # Process tweets
                            if page_response_obj.data:
                                for tweet_obj_data_item in page_response_obj.data:
                                    if tweets_for_account >= account_limit or total_tweets_downloaded_run >= user_confirmed_download_limit_global:
                                        break

                                    # Prepare attachment summary with details
                                    attachment_summary_dict = {}
                                    if 'attachments' in tweet_obj_data_item.data:
                                        if 'media_keys' in tweet_obj_data_item.data['attachments']:
                                            attachment_summary_dict['media_present'] = True
                                            attachment_summary_dict['media_keys'] = tweet_obj_data_item.data['attachments']['media_keys']
                                            media_details = []
                                            for media_key in tweet_obj_data_item.data['attachments']['media_keys']:
                                                for media_item in serializable_includes.get('media', []):
                                                    if media_item.get('media_key') == media_key:
                                                        media_details.append({
                                                            'media_key': media_key,
                                                            'type': media_item.get('type'),
                                                            'url': media_item.get('url'),
                                                            'preview_image_url': media_item.get('preview_image_url'),
                                                            'alt_text': media_item.get('alt_text'),
                                                            'variants': media_item.get('variants', [])
                                                        })
                                            if media_details:
                                                attachment_summary_dict['media_details'] = media_details

                                        if 'poll_ids' in tweet_obj_data_item.data['attachments']:
                                            attachment_summary_dict['poll_present'] = True
                                            attachment_summary_dict['poll_ids'] = tweet_obj_data_item.data['attachments']['poll_ids']
                                            poll_details = []
                                            for poll_id in tweet_obj_data_item.data['attachments']['poll_ids']:
                                                for poll_item in serializable_includes.get('polls', []):
                                                    if poll_item.get('id') == poll_id:
                                                        poll_details.append({
                                                            'poll_id': poll_id,
                                                            'options': poll_item.get('options'),
                                                            'voting_status': poll_item.get('voting_status'),
                                                            'end_datetime': poll_item.get('end_datetime')
                                                        })
                                            if poll_details:
                                                attachment_summary_dict['poll_details'] = poll_details

                                    # Обработка context_annotations
                                    context_annotations_summary = []
                                    if 'context_annotations' in tweet_obj_data_item.data:
                                        for annotation in tweet_obj_data_item.data['context_annotations']:
                                            context_annotations_summary.append({
                                                'domain': annotation.get('domain', {}),
                                                'entity': annotation.get('entity', {})
                                            })

                                    current_tweet_data_dict = tweet_obj_data_item.data.copy()
                                    if attachment_summary_dict:
                                        current_tweet_data_dict['attachment_summary'] = attachment_summary_dict
                                    if context_annotations_summary:
                                        current_tweet_data_dict['context_annotations_summary'] = context_annotations_summary

                                    serializable_meta = {}
                                    if hasattr(page_response_obj, 'meta') and page_response_obj.meta:
                                        serializable_meta = dict(page_response_obj.meta)

                                    # Добавляем username автора для удобства
                                    if 'author_id' in current_tweet_data_dict and serializable_includes.get('users'):
                                        author_id = current_tweet_data_dict['author_id']
                                        for user in serializable_includes['users']:
                                            if user.get('id') == author_id:
                                                current_tweet_data_dict['author_username'] = user.get('username', 'unknown')
                                                current_tweet_data_dict['author_name'] = user.get('name', 'unknown')
                                                break

                                    # Сохраняем твит с полными данными
                                    tweet_to_save_dict = {
                                        "tweet_data": current_tweet_data_dict,
                                        "includes": serializable_includes,
                                        "meta_on_fetch": serializable_meta
                                    }

                                    tweet_id_val_str = str(tweet_obj_data_item.id)
                                    # Создаем директорию только при сохранении первого твита
                                    if not directory_created:
                                        os.makedirs(save_directory_path, exist_ok=True)
                                        directory_created = True

                                    tweet_json_file_path = os.path.join(save_directory_path, f"{tweet_id_val_str}.json")

                                    with open(tweet_json_file_path, 'w', encoding='utf-8') as f_tweet_json_out:
                                        json.dump(tweet_to_save_dict, f_tweet_json_out, ensure_ascii=False, indent=2)

                                    tweets_for_account += 1
                                    total_tweets_downloaded_run += 1
                                    total_tweets_for_batch += 1
                                    # Обновляем прогресс-бар
                                    pbar_overall_dl_tasks.update(1)
                                    pbar_overall_dl_tasks.set_postfix_str(f"{total_tweets_downloaded_run}/{total_tweets_to_download} tweets")

                                    # Собираем статистику
                                    account_clean = account.lstrip('@')
                                    if account_clean not in download_statistics:
                                        download_statistics[account_clean] = {}
                                    tweet_type = current_task_series_data['Tweet_Type_Desc']
                                    if tweet_type not in download_statistics[account_clean]:
                                        download_statistics[account_clean][tweet_type] = 0
                                    download_statistics[account_clean][tweet_type] += 1

                                    # Обновляем статус каждые 10 твитов
                                    current_status['tweets_downloaded'] = total_tweets_downloaded_run
                                    if total_tweets_downloaded_run % 10 == 0:
                                        update_download_status()

                            time.sleep(FIXED_PAUSE_SECONDS)

                    except tweepy.TooManyRequests as tmr_dl:
                        update_status_display_fn([f"RL on {token_id_dl_used_str} during batch DL for {account}."])
                        reset_unix_dl = tmr_dl.response.headers.get('x-rate-limit-reset')
                        reset_dt_dl = datetime.fromtimestamp(int(reset_unix_dl), timezone.utc) if reset_unix_dl else datetime.now(timezone.utc) + timedelta(minutes=16)
                        clients_manager[token_id_dl_used_str]['rate_limited_until'] = reset_dt_dl + timedelta(seconds=10)
                        update_client_rate_limit_info_fn(token_id_dl_used_str, tmr_dl.response.headers, f"dl_batch_tmr_{account[:10]}")
                        update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
                        log_detailed_event('rate_limit_events', {
                            'token': token_id_dl_used_str,
                            'account': account,
                            'reset_time': reset_dt_dl.isoformat()
                        })
                    except Exception as e:
                        error_msg = f"Error processing account {account}: {str(e)[:100]}"
                        update_status_display_fn([error_msg])
                        log_detailed_event('errors_log', {
                            'account': account,
                            'error': str(e),
                            'traceback': traceback.format_exc()
                        })
                        traceback.print_exc()
                        continue

                # Логируем результат batch
                download_log_list.append({
                    "task_details": current_task_series_data.to_dict(),
                    "attempted_to_fetch": current_task_series_data['Estimated_Count'],
                    "actually_fetched": total_tweets_for_batch,
                    "tweets_by_account": {acc: "processed" for acc in accounts_in_batch},
                    "status": f"Batch processed: {total_tweets_for_batch} tweets from {len(accounts_in_batch)} accounts",
                    "api_pages_fetched": "multiple"
                })

                update_status_display_fn([f"Batch complete: {total_tweets_for_batch} tweets from {len(accounts_in_batch)} accounts ({current_task_series_data['Tweet_Type_Desc']})"])

            else:
                # ОБЫЧНАЯ ЗАДАЧА (не batch) - используем старую логику
                current_status['total_accounts'] = 1
                current_status['accounts_processed'] = 1
                current_status['current_account'] = current_task_series_data['Account']
                current_status['current_type'] = current_task_series_data['Tweet_Type_Desc']

                limit_for_this_task = min(current_task_series_data['Estimated_Count'], user_confirmed_download_limit_global - total_tweets_downloaded_run)
                if limit_for_this_task <= 0:
                    pbar_overall_dl_tasks.update(1)
                    continue

                fimi_slug_path = get_fimi_slug(current_task_series_data['FIMI_Event'])
                save_directory_path = os.path.join(ACTORS_BASE_DIR, fimi_slug_path, current_task_series_data['Account'], current_task_series_data['Query_Tag_For_Path'], current_task_series_data['Tweet_Type_Slug_For_Path'])
                # НЕ создаем директорию здесь
                directory_created_for_task = False

                tweets_downloaded_for_this_task = 0
                estimated_count_for_task = current_task_series_data['Estimated_Count']

                update_status_display_fn([f"Fetching up to {limit_for_this_task} for {current_task_series_data['Account']}... (Est: {estimated_count_for_task})"])

                task_retries_count = 0
                max_task_retries_allowed = len(clients_manager) + 2
                pagination_token_for_task_retry = None
                pages_fetched_for_task = 0

                while tweets_downloaded_for_this_task < limit_for_this_task and task_retries_count < max_task_retries_allowed:
                    if total_tweets_downloaded_run >= user_confirmed_download_limit_global: break

                    active_dl_client_inst, token_id_dl_used_str = get_next_available_client_fn()
                    if not active_dl_client_inst:
                        update_status_display_fn(["No client available for DL task, breaking task."])
                        break

                    if task_retries_count > 0:
                        update_status_display_fn([f"Retrying {task_short_name} with {token_id_dl_used_str} (Attempt {task_retries_count + 1})..."])

                    remaining_for_task_in_this_attempt = int(limit_for_this_task - tweets_downloaded_for_this_task)
                    if remaining_for_task_in_this_attempt <= 0: break

                    try:
                        # Фиксированная пауза
                        time.sleep(FIXED_PAUSE_SECONDS)

                        tweet_paginator_instance = tweepy.Paginator(
                            active_dl_client_inst.search_all_tweets,
                            query=current_task_series_data['Actual_API_Query_Used'],
                            start_time=current_task_series_data['Start_Time_API'],
                            end_time=current_task_series_data['End_Time_API'],
                            max_results=api_page_max_results,
                            tweet_fields=TWEET_FIELDS_COMPREHENSIVE,
                            expansions=EXPANSIONS_COMPREHENSIVE,
                            user_fields=USER_FIELDS_COMPREHENSIVE,
                            media_fields=MEDIA_FIELDS_COMPREHENSIVE,
                            poll_fields=POLL_FIELDS_COMPREHENSIVE,
                            place_fields=PLACE_FIELDS_COMPREHENSIVE,
                            next_token=pagination_token_for_task_retry
                        )

                        pagination_token_for_task_retry = None

                        with tqdm(total=remaining_for_task_in_this_attempt, desc=f"{current_task_series_data['Account']} ({token_id_dl_used_str})", leave=False, unit="tw", position=1) as pbar_tweets_in_task_ui:
                            for page_response_obj in tweet_paginator_instance:
                                pages_fetched_for_task += 1

                                # Extract headers correctly
                                page_headers = {}
                                if hasattr(page_response_obj, '_headers'):
                                    page_headers = dict(page_response_obj._headers)
                                elif hasattr(page_response_obj, 'response') and hasattr(page_response_obj.response, 'headers'):
                                    page_headers = dict(page_response_obj.response.headers)

                                update_client_rate_limit_info_fn(token_id_dl_used_str, page_headers, f"dl_search_page_{task_short_name[:10]}")

                                pagination_token_for_task_retry = page_response_obj.meta.get('next_token') if page_response_obj.meta else None

                                # Convert includes to serializable format
                                serializable_includes = {}
                                if hasattr(page_response_obj, 'includes') and page_response_obj.includes:
                                    if hasattr(page_response_obj.includes, 'users') and page_response_obj.includes.users:
                                        serializable_includes['users'] = [user.data for user in page_response_obj.includes.users]
                                    if hasattr(page_response_obj.includes, 'media') and page_response_obj.includes.media:
                                        serializable_includes['media'] = [media.data for media in page_response_obj.includes.media]
                                    if hasattr(page_response_obj.includes, 'polls') and page_response_obj.includes.polls:
                                        serializable_includes['polls'] = [poll.data for poll in page_response_obj.includes.polls]
                                    if hasattr(page_response_obj.includes, 'places') and page_response_obj.includes.places:
                                        serializable_includes['places'] = [place.data for place in page_response_obj.includes.places]
                                    if hasattr(page_response_obj.includes, 'tweets') and page_response_obj.includes.tweets:
                                        serializable_includes['tweets'] = [tweet.data for tweet in page_response_obj.includes.tweets]

                                if page_response_obj.data:
                                    for tweet_obj_data_item in page_response_obj.data:
                                        if tweets_downloaded_for_this_task >= limit_for_this_task or total_tweets_downloaded_run >= user_confirmed_download_limit_global: break

                                        # Prepare attachment summary with details
                                        attachment_summary_dict = {}
                                        if 'attachments' in tweet_obj_data_item.data:
                                            if 'media_keys' in tweet_obj_data_item.data['attachments']:
                                                attachment_summary_dict['media_present'] = True
                                                attachment_summary_dict['media_keys'] = tweet_obj_data_item.data['attachments']['media_keys']
                                                media_details = []
                                                for media_key in tweet_obj_data_item.data['attachments']['media_keys']:
                                                    for media_item in serializable_includes.get('media', []):
                                                        if media_item.get('media_key') == media_key:
                                                            media_details.append({
                                                                'media_key': media_key,
                                                                'type': media_item.get('type'),
                                                                'url': media_item.get('url'),
                                                                'preview_image_url': media_item.get('preview_image_url'),
                                                                'alt_text': media_item.get('alt_text'),
                                                                'variants': media_item.get('variants', [])
                                                            })
                                                if media_details:
                                                    attachment_summary_dict['media_details'] = media_details

                                            if 'poll_ids' in tweet_obj_data_item.data['attachments']:
                                                attachment_summary_dict['poll_present'] = True
                                                attachment_summary_dict['poll_ids'] = tweet_obj_data_item.data['attachments']['poll_ids']
                                                poll_details = []
                                                for poll_id in tweet_obj_data_item.data['attachments']['poll_ids']:
                                                    for poll_item in serializable_includes.get('polls', []):
                                                        if poll_item.get('id') == poll_id:
                                                            poll_details.append({
                                                                'poll_id': poll_id,
                                                                'options': poll_item.get('options'),
                                                                'voting_status': poll_item.get('voting_status'),
                                                                'end_datetime': poll_item.get('end_datetime')
                                                            })
                                                if poll_details:
                                                    attachment_summary_dict['poll_details'] = poll_details

                                        # Обработка context_annotations
                                        context_annotations_summary = []
                                        if 'context_annotations' in tweet_obj_data_item.data:
                                            for annotation in tweet_obj_data_item.data['context_annotations']:
                                                context_annotations_summary.append({
                                                    'domain': annotation.get('domain', {}),
                                                    'entity': annotation.get('entity', {})
                                                })

                                        current_tweet_data_dict = tweet_obj_data_item.data.copy()
                                        if attachment_summary_dict:
                                            current_tweet_data_dict['attachment_summary'] = attachment_summary_dict
                                        if context_annotations_summary:
                                            current_tweet_data_dict['context_annotations_summary'] = context_annotations_summary

                                        serializable_meta = {}
                                        if hasattr(page_response_obj, 'meta') and page_response_obj.meta:
                                            serializable_meta = dict(page_response_obj.meta)

                                        tweet_to_save_dict = {
                                            "tweet_data": current_tweet_data_dict,
                                            "includes": serializable_includes,
                                            "meta_on_fetch": serializable_meta
                                        }

                                        tweet_id_val_str = str(tweet_obj_data_item.id)
                                        # Создаем директорию только при первом сохранении
                                        if not directory_created_for_task:
                                            try:
                                                os.makedirs(save_directory_path, exist_ok=True)
                                                directory_created_for_task = True
                                            except Exception as e_mkdir_dl:
                                                update_status_display_fn([f"Err mkdir DL {save_directory_path}: {str(e_mkdir_dl)[:30]}"])
                                                continue

                                        tweet_json_file_path = os.path.join(save_directory_path, f"{tweet_id_val_str}.json")
                                        with open(tweet_json_file_path, 'w', encoding='utf-8') as f_tweet_json_out:
                                            json.dump(tweet_to_save_dict, f_tweet_json_out, ensure_ascii=False, indent=2)

                                        tweets_downloaded_for_this_task += 1
                                        total_tweets_downloaded_run += 1
                                        pbar_tweets_in_task_ui.update(1)

                                        # Обновляем общий прогресс-бар
                                        pbar_overall_dl_tasks.update(1)
                                        pbar_overall_dl_tasks.set_postfix_str(f"{total_tweets_downloaded_run}/{total_tweets_to_download} tweets")

                                        # Собираем статистику
                                        account_clean = current_task_series_data['Account'].lstrip('@')
                                        if account_clean == "GLOBAL_SEARCH":
                                            account_clean = "GLOBAL_SEARCH"
                                        if account_clean not in download_statistics:
                                            download_statistics[account_clean] = {}
                                        tweet_type = current_task_series_data['Tweet_Type_Desc']
                                        if tweet_type not in download_statistics[account_clean]:
                                            download_statistics[account_clean][tweet_type] = 0
                                        download_statistics[account_clean][tweet_type] += 1

                                        # Обновляем общий статус
                                        current_status['tweets_downloaded'] = total_tweets_downloaded_run
                                        if total_tweets_downloaded_run % 10 == 0:
                                            update_download_status()

                                        pbar_tweets_in_task_ui.set_postfix_str(f"Total: {total_tweets_downloaded_run}/{user_confirmed_download_limit_global} | Task: {tweets_downloaded_for_this_task}/{estimated_count_for_task} | Pause: {FIXED_PAUSE_SECONDS}s")

                                if tweets_downloaded_for_this_task >= limit_for_this_task or total_tweets_downloaded_run >= user_confirmed_download_limit_global: break

                                # Фиксированная пауза между страницами
                                time.sleep(FIXED_PAUSE_SECONDS)

                        # Успешно завершили - выходим из retry цикла
                        break

                    except tweepy.TooManyRequests as tmr_dl:
                        update_status_display_fn([f"RL on {token_id_dl_used_str} during DL for {task_short_name}."])
                        reset_unix_dl = tmr_dl.response.headers.get('x-rate-limit-reset')
                        reset_dt_dl = datetime.fromtimestamp(int(reset_unix_dl), timezone.utc) if reset_unix_dl else datetime.now(timezone.utc) + timedelta(minutes=16)
                        clients_manager[token_id_dl_used_str]['rate_limited_until'] = reset_dt_dl + timedelta(seconds=10)
                        update_client_rate_limit_info_fn(token_id_dl_used_str, tmr_dl.response.headers, f"dl_search_tmr_{task_short_name[:10]}")
                        update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
                        task_retries_count += 1
                        if task_retries_count >= max_task_retries_allowed:
                            update_status_display_fn([f"Max retries for {task_short_name}. Moving to next task."])
                            break
                    except Exception as e_dl:
                        update_status_display_fn([f"DL Err {task_short_name} w/{token_id_dl_used_str}: {str(e_dl)[:50]}"])
                        traceback.print_exc()
                        break

                status_for_log = "OK"
                if tweets_downloaded_for_this_task < limit_for_this_task and total_tweets_downloaded_run < user_confirmed_download_limit_global:
                    status_for_log = f"Partial ({tweets_downloaded_for_this_task}/{limit_for_this_task} due to error/limit or no more data)"
                elif tweets_downloaded_for_this_task == 0:
                    status_for_log = "NoTweetsFetched_Or_Error"

                download_log_list.append({
                    "task_details": current_task_series_data.to_dict(),
                    "attempted_to_fetch_for_task": limit_for_this_task,
                    "actually_fetched_for_task": tweets_downloaded_for_this_task,
                    "save_directory": save_directory_path,
                    "status": status_for_log,
                    "estimated_total_for_task": estimated_count_for_task,
                    "api_pages_fetched": pages_fetched_for_task
                })
                update_status_display_fn([f"Task {current_task_series_data['Account'][:10]}: Downloaded {tweets_downloaded_for_this_task}/{estimated_count_for_task} (Est) in {pages_fetched_for_task} API calls. Total run: {total_tweets_downloaded_run}."])

            # Не обновляем прогресс-бар здесь, так как теперь обновляем при каждом твите
            if total_tweets_downloaded_run >= user_confirmed_download_limit_global: break

    # Phase 4: Final token usage summary
    token_usage_summary = {token_id: info['request_count'] for token_id, info in clients_manager.items()}

    # Формируем красивый вывод статистики
    print("\n" + "="*60)
    print("DOWNLOAD SUMMARY")
    print("="*60)
    print(f"Total tweets downloaded: {total_tweets_downloaded_run}")
    print("\nBreakdown by account and type:")
    print("-"*60)

    for account, types in sorted(download_statistics.items()):
        account_total = sum(types.values())
        print(f"\n{account}: {account_total} total")
        for tweet_type, count in sorted(types.items()):
            print(f"  - {tweet_type}: {count}")

    print("\n" + "="*60 + "\n")

    # Обновляем статус с итоговой информацией
    summary_lines = [f"Phase 4 DL complete. Total: {total_tweets_downloaded_run} tweets"]
    for account, types in sorted(download_statistics.items()):
        account_total = sum(types.values())
        summary_lines.append(f"{account}: {account_total} tweets")
    update_status_display_fn(summary_lines)

    # Добавляем статистику в логи
    log_dl_complete_data = {
        "stage": "dl_complete_ph4",
        "ts_utc": datetime.now(timezone.utc).isoformat(timespec='seconds').replace('+00:00', 'Z'),
        "user_conf_limit": user_confirmed_download_limit_global,
        "total_dl_this_run": total_tweets_downloaded_run,
        "download_statistics": download_statistics,  # Новое поле со статистикой
        "dl_summary_per_task": download_log_list,
        "token_usage_final": token_usage_summary
    }
    log_operation_fn(log_dl_complete_data, "dl_complete_ph4")
    download_button_ui.disabled = True
    estimation_output_area_ui.clear_output()
    download_confirmation_dialog_output_ui.clear_output()

@reset_button_ui.on_click
def handle_reset_button_click_fn(b_reset_ui):
    global estimation_results_df, user_id_cache, user_confirmed_download_limit_global
    global task_selection_checkboxes_global, _status_lines, clients_manager
    global _current_active_token, estimated_requests_per_task

    # 1. Сброс всех полей ввода
    account_names_input.value = 'XDevelopers,elonmusk'
    search_queries_input.value = ''
    fimi_dropdown.value = FIMI_EVENTS[0]
    start_date_picker.value = default_start_date_val
    end_date_picker.value = default_end_date_val

    # 2. Сброс чек-боксов типов твитов
    for cb_item_val in tweet_type_checkboxes_map_ui.values():
        cb_item_val.value = True
    select_all_tweet_types_cb_ui.value = True

    # 3. Сброс чек-боксов выбора токенов  ← **исправленный блок**
    for cb in token_selection_checkboxes.values():
        cb.value = True

    # 4. Остальные переключатели
    rate_limit_optimization_checkbox.value = True  # Phase 4

    # 5. Очистка состояний и интерфейса
    status_html.value = "<i>Form reset. Configure and estimate.</i>"
    _status_lines.clear()
    update_status_display_fn(["Form reset."])

    estimation_output_area_ui.clear_output()
    download_confirmation_dialog_output_ui.clear_output()
    download_button_ui.disabled = True

    # 6. Сброс внутренних данных
    estimation_results_df = pd.DataFrame()
    user_id_cache.clear()
    user_confirmed_download_limit_global = 0
    user_limit_input_widget_ui.value = 1000
    task_selection_checkboxes_global.clear()
    _current_active_token = None          # Phase 4
    estimated_requests_per_task.clear()   # Phase 4

    # 7. Сброс счётчиков rate-limit по токенам
    if clients_manager:
        for token_id_key_val in clients_manager:
            clients_manager[token_id_key_val]['rate_limited_until'] = (
                datetime.now(timezone.utc) - timedelta(seconds=1)
            )
            clients_manager[token_id_key_val]['request_count'] = 0  # Phase 4

    # 8. Обновление статуса
    update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
    update_status_display_fn(["Client rate limit statuses reset (available)."])


# --- UI Layout and Display ---
input_section_ui_layout = VBox([
    widgets.HTML("<h3>1. Configure Data Collection Parameters:</h3>"),
    token_selection_box,  # Добавлено
    account_names_input,
    search_queries_input,
    fimi_dropdown,
    widgets.HTML("<b>Select Date Range (UTC):</b>"),
    date_box,
    widgets.HTML("<b>Select Tweet Types & Options:</b>"),
    tweet_types_box_layout_ui
], layout=Layout(border='1px solid #B0BEC5', padding='15px', margin='10px', width='auto'))
actions_section_ui_layout = VBox([widgets.HTML("<h3>2. Execute Actions:</h3>"), action_buttons_box_ui, widgets.HTML("<b>Live Status Updates (Phase 4 - Color Coded):</b>"), status_html], layout=Layout(border='1px solid #B0BEC5', padding='15px', margin='10px', width='auto'))
results_section_ui_layout = VBox([widgets.HTML("<h3>3. Estimation Results & Download Confirmation:</h3>"), estimation_output_area_ui, download_confirmation_dialog_output_ui], layout=Layout(border='1px solid #B0BEC5', padding='15px', margin='10px', width='auto'))
ui_main_layout = VBox([input_section_ui_layout, actions_section_ui_layout, results_section_ui_layout])

if clients_fully_initialized_flag and clients_manager and any(clients_manager):
    display(ui_main_layout)
    update_status_display_fn(["Phase 4 Interface loaded. Smart rate limiting enabled.", "Configure parameters and click 'Estimate'."])
    update_status_display_fn(_get_token_status_string_fn(), is_token_status_update=True)
else:
    display(IPHTML("<h2><span style='color:red;'>CRITICAL ERROR: NO Tweepy clients initialized.</span></h2><p>Token files missing/invalid or path issue. Check `KEYS_PATH` and token files. Re-run cell.</p>"))